In [ ]:
import csv
import cv2
import numpy as np

csv_file = './DATA/driving_log.csv'

lines = []
with open(csv_file) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './DATA/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)

X_train = np.array(images)
y_train = np.array(measurements)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

model_name = "linear_regx"

model = Sequential()
#model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Flatten(input_shape=(160,320,3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

model.save('models/{}/model.h5'.format(model_name))

In [ ]:
model_name = "linear_regx_norm"

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=1)

model.save('models/{}/model.h5'.format(model_name))

In [ ]:
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model_name = "lenet"

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=4)

model.save('models/{}/model.h5'.format(model_name))

In [ ]:
augmented_images, augmented_measurments = [], []
for image, measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurments.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurments.append(measurement*-1.0)

X_train = np.array(augmented_images)
y_train = np.array(augmented_measurments)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
model_name = "lenet_augmented"

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=4)

model.save('models/{}/model.h5'.format(model_name))

In [3]:
import csv
import cv2
import numpy as np

csv_file = './DATA/driving_log.csv'

lines = []
with open(csv_file) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
for line in lines:
    def process_image(path):
        filename = path.split('/')[-1]
        current_path = './DATA/IMG/' + filename
        return cv2.imread(current_path)
    
    source_path_center = line[0]
    source_path_left = line[1]
    source_path_right = line[2]
    
    images.append(process_image(source_path_center))
    images.append(process_image(source_path_left))
    images.append(process_image(source_path_right))
        
    # create adjusted steering measurements for the side camera images
    correction = 0.25
    measurement_center = float(line[3])
    measurement_left = measurement_center + correction
    measurement_right = measurement_center - correction
    
    measurements.append(measurement_center)
    measurements.append(measurement_left)
    measurements.append(measurement_right)

augmented_images, augmented_measurments = [], []
for image, measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurments.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurments.append(measurement*-1.0)

X_train = np.array(augmented_images)
y_train = np.array(augmented_measurments)

In [18]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model_name = "lenet_augmented_3cams"

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5,subsample=(2,2),activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dropout(0.5))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=1)

model.save('models/{}/model.h5'.format(model_name))

Train on 33331 samples, validate on 8333 samples
Epoch 1/1
33331/33331 [==============================] - 70s - loss: 0.0383 - val_loss: 0.0180


In [47]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model_name = "lenet_augmented_3cams_cropped"

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,25),(0,0))))
model.add(MaxPooling2D())
model.add(Convolution2D(6,8,8,subsample=(2,2),activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2)

model.save('models/{}/model.h5'.format(model_name))

Train on 33331 samples, validate on 8333 samples
Epoch 1/2
33331/33331 [==============================] - 95s - loss: 0.0333 - val_loss: 0.0294
Epoch 2/2
33331/33331 [==============================] - 97s - loss: 0.0244 - val_loss: 0.0378


In [40]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model_name = "nvidia_augmented_3cams_cropped"

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,25),(0,0))))
model.add(Convolution2D(16, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2)

model.save('models/{}/model.h5'.format(model_name))

Train on 33331 samples, validate on 8333 samples
Epoch 1/2
33331/33331 [==============================] - 165s - loss: 0.0347 - val_loss: 0.0415
Epoch 2/2
33331/33331 [==============================] - 172s - loss: 0.0220 - val_loss: 0.0493


In [75]:
import os
import csv
import cv2
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

csv_file = './DATA/driving_log.csv'

samples = []
with open(csv_file) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, batch_size=32, train=False):
    angle_correction = [0., +.2, -.2]
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                for i in range(3):
                    name = './DATA/IMG/'+batch_sample[i].split('/')[-1]
                    image = cv2.imread(name)
                    angle = float(batch_sample[3]) + angle_correction[i]
                    images.append(image)
                    angles.append(angle)
                
                augmented_images, augmented_angles = [], []
                for image, angle in zip(images,angles):
                    augmented_images.append(image)
                    augmented_angles.append(angle)
                    augmented_images.append(cv2.flip(image,1))
                    augmented_angles.append(angle*-1.0)
                    
            def sampleLists(X,y, sample=0.5):
                X_shuff, y_shuff = sklearn.utils.shuffle(X,y)
                size = int(len(X)*sample)
                return np.array(X_shuff[:size]), np.array(y_shuff[:size])

            sample = 0.5 if train else 1.0
            yield sampleLists(augmented_images, augmented_angles, sample=sample)

batch_size = 32
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size, train=True)
validation_generator = generator(validation_samples, batch_size=batch_size)

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,subsample=(2,2),activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dropout(0.5))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='nadam')
model.fit_generator(train_generator, samples_per_epoch=len(train_samples)*3,
                    validation_data=validation_generator, nb_val_samples=len(validation_samples)*6,
                    nb_epoch=4)

Epoch 1/4
16665/16665 [==============================] - 33s - loss: 0.0463 - val_loss: 0.0361
Epoch 2/4
16665/16665 [==============================] - 32s - loss: 0.0386 - val_loss: 0.0340
Epoch 3/4
16665/16665 [==============================] - 32s - loss: 0.0371 - val_loss: 0.0336
Epoch 4/4
16665/16665 [==============================] - 32s - loss: 0.0348 - val_loss: 0.0329


In [ ]:
model.save('./model.h5')